In [17]:
import typing
from decimal import Decimal
from enum import Enum

import copy

def to_string(blob: typing.Union[dict,str,int], tabs_level: int = 0, brackets = False, prev_list: bool = False) -> str:
    # print(blob, tabs_level)
    tabs = '\t' * tabs_level
    prev_tabs = '\t' * (tabs_level - 1)
    if isinstance(blob, dict):
        # print("!!!")
        # list_of_things =
        list_of_things = [f'{tabs}{key} = {to_string(blob[key], tabs_level + 1, True)}' for key in blob]
        meat = '\n'.join(list_of_things)
        if not brackets and not prev_list:
            return f'{meat}'
        else:
            return f'{prev_tabs}{{\n{meat}\n{prev_tabs}}}'
    if isinstance(blob, list):
        list_of_things = [f'{to_string(item, tabs_level, prev_list=True)}' for item in blob]
        meat = '\n'.join(list_of_things)
        if not brackets:
            return f'{meat}'
        else:
            return f'{{\n{meat}\n{prev_tabs}}}'
    else:
        if prev_list:
            return tabs + str(blob)
        else:
            return str(blob)

def dicts_to_str(items: list[dict]):
    result = ''
    for item in items:
        result += str(item)
    return result

In [18]:
CACHE_PM = []
CACHE_PM_ARTISAN = []

class ProductionMethod():
    def __init__(self, name, dict) -> None:
        self.dict = dict
        self.name = name

        dict[name]['value']

        CACHE_PM.append(self)

        artisan_dict = {
            "artisan_" + name: copy.deepcopy(dict[name])
        }

        artisan_dict["artisan_" + name]['owner'] = {
            'poptype': 'artisans',
            'effect': 'output'
        }
        artisan_dict["artisan_" + name]['type'] = 'artisan'
        artisan_dict["artisan_" + name]['workforce'] = 10000
        artisan_dict["artisan_" + name].pop('template', None)

        CACHE_PM_ARTISAN.append(artisan_dict)

    def __str__(self):
        return self.name

def print_pms():
    for item in CACHE_PM:
        print(to_string(item.dict))

def print_artisans():
    for item in CACHE_PM_ARTISAN:
        print(to_string(item))

In [19]:
CACHE_PTT = []

class PT_Template():
    def __init__(self, name, dict: dict):
        self.dict = dict
        self.name = name

        CACHE_PTT.append(self)

    def __str__(self) -> str:
        return self.name

def print_pt_templates():
    for item in CACHE_PTT:
        print(to_string(item.dict))

In [20]:
def pt_template_capitalists(name, efficiency):
    return PT_Template(name,
        {
            name: {
                'efficiency': efficiency,
                'owner': {
                    'poptype': 'capitalists',
                    'effect': 'input',
                    'effect_multiplier': -2.5
                },
                'employees': [
                    {
                        'poptype': 'craftsmen',
                        'effect': 'throughput',
                        'amount': 0.8
                    },
                    {
                        'poptype': 'clerks',
                        'effect': 'output',
                        'effect_multiplier': 1.25,
                        'amount': 0.2
                    }
                ],
                'type': 'factory',
                'workforce': 10000
            }
        }
    )

In [21]:

PRODUCTION_MULTIPLIER = 10

def pm(name: str, template:PT_Template, inputs: dict, output: str, value: int):
    for key in inputs.keys():
        inputs[key] *= PRODUCTION_MULTIPLIER

    return ProductionMethod(name,
        {
            name: {
                'template': template,
                'input_goods': inputs,
                'output_goods': output,
                'value': value * PRODUCTION_MULTIPLIER
            }
        }
    )

In [22]:
GLOBAL_BUILDING_TIME = 30
CACHE_FACTORY = []

def factory(name: str, cost: dict, enabled: str, production_method: ProductionMethod):
    global GLOBAL_BUILDING_TIME

    factory = {
        name: {
            'type': 'factory',
            'on_completion': 'factory',
            'completion_size': 0.2,
            'max_level': 256,
            'goods_cost': cost,
            'time': GLOBAL_BUILDING_TIME,
            'visibility': 'yes',
            'onmap': 'no',
            'default_enabled': enabled,
            'production_type': production_method.name,
            'pop_build_factory': 'yes'
        }
    }

    CACHE_FACTORY.append(factory)

def coastal_factory(name: str, cost: dict, enabled: str, production_method: ProductionMethod):
    global GLOBAL_BUILDING_TIME

    factory = {
        name: {
            'type': 'factory',
            'on_completion': 'factory',
            'completion_size': 0.2,
            'max_level': 256,
            'goods_cost': cost,
            'time': GLOBAL_BUILDING_TIME,
            'visibility': 'yes',
            'onmap': 'no',
            'default_enabled': enabled,
            'production_type': production_method.name,
            'pop_build_factory': 'yes',
            'is_coastal': 'yes'
        }
    }

    CACHE_FACTORY.append(factory)

def print_factories():
    for item in CACHE_FACTORY:
        print(to_string(item))

In [23]:
ammunition = 'ammunition'
small_arms = 'small_arms'
artillery = 'artillery'
canned_food = 'canned_food'
tanks = 'barrels'
aero = 'aeroplanes'
military_goods = [ammunition, small_arms, artillery, canned_food, tanks, aero]

cotton = 'cotton'
dye = 'dye'
wool = 'wool'
silk = 'silk'
coal = 'coal'
sulphur = 'sulphur'
iron = 'iron'
timber = 'timber'
tropical_wood = 'tropical_wood'
rubber = 'rubber'
oil = 'oil'
fuel = 'fuel'
precious_metal = 'precious_metal'

steel = 'steel'
cement = 'cement'
machine_parts = 'machine_parts'
explosives = 'explosives'
lumber = 'lumber'
fertilizer = 'fertilizer'

steamer_convoy = 'steamer_convoy'
clipper_convoy = 'clipper_convoy'

fabric = 'fabric'
regular_clothes = 'regular_clothes'
luxury_clothes = 'luxury_clothes'

furniture = 'furniture'
luxury_furniture = 'luxury_furniture'

paper = 'paper'
glass = 'glass'

cattle = 'cattle'
fruit = 'fruit'
fish = 'fish'
grain = 'grain'

liquor = 'liquor'
wine = 'wine'

electric_gear = 'electric_gear'
radio = 'radio'
automobiles = 'automobiles'
barrels = 'barrels'
telephones = 'telephones'

In [24]:
fabric_inputs = [wool, cotton, silk]
canned_food_inputs = [cattle, fish]
lumber_inputs = [timber, tropical_wood]

In [25]:
def pm_generate_name(output, main_input):
    if output == 'fabric':
        name = 'fabric_factory'
        if main_input != 'cotton':
            name += '_' + main_input
    if output == 'canned_food':
        name = 'canned_food_factory'
        if main_input != cattle:
            name += '_' + main_input
    if output == 'lumber':
        name = 'lumber_mill'
        if main_input != timber:
            name += "_" + main_input

    return name

In [26]:
primitive_factory_cost = {
    iron: 10,
    timber: 10
}

basic_factory_cost = {
    cement: 10,
    machine_parts: 10,
}

advanced_factory_cost = {
    cement: 10,
    machine_parts: 10,
    steel: 10
}

In [27]:
fabric_template = pt_template_capitalists(
    'factory_dye',
    {
        cement: 0.5,
        machine_parts: 0.05,
        dye: 2.0,
    }
)

basic_template = pt_template_capitalists(
    'factory_template',
    {
        cement: 0.5,
        machine_parts: 0.05,
    }
)

primitive_template = pt_template_capitalists(
    'cheap_factory_template',
    {
        iron: 0.5,
        timber: 0.5
    }
)

In [28]:
for possible_input in fabric_inputs:
    name = pm_generate_name(fabric, possible_input)
    production_method = pm(
        name,
        fabric_template,
        {
            possible_input: 1
        },
        fabric,
        1
    )
    factory(
        name,
        basic_factory_cost,
        'no',
        production_method
    )

for possible_input in canned_food_inputs:
    name = pm_generate_name(canned_food, possible_input)
    production_method = pm(
        name,
        basic_template,
        {
            possible_input: 1,
            iron: 1
        },
        canned_food,
        1
    )
    factory(
        name,
        basic_factory_cost,
        'no',
        production_method
    )

for possible_input in lumber_inputs:
    name = pm_generate_name(lumber, possible_input)
    factory(name, basic_factory_cost, 'no', pm(
        name, basic_template,
        {
            possible_input: 1,
        },
        lumber, 1
    ))


In [29]:

### LATE GAME FACTORIES:
name = 'aeroplane_factory'
aeroplane_pm = pm(
    name,
    basic_template,
    {
        machine_parts: 0.4,
        electric_gear: 0.4,
        rubber: 0.1,
        lumber: 0.1
    },
    aero,
    1
)
factory(name, advanced_factory_cost, 'no', aeroplane_pm)

name = 'barrel_factory'
barrel_pm = pm(
    name,
    basic_template,
    {
        automobiles: 0.3,
        machine_parts: 0.3,
        electric_gear: 0.1,
        steel: 0.3
    },
    barrels,
    1
)
factory(name, advanced_factory_cost, 'no', barrel_pm)

name = 'automobile_factory'
temp = pm(
    name,
    basic_template,
    {
        machine_parts: 0.3,
        electric_gear: 0.3,
        rubber: 0.2,
        steel: 0.2
    },
    automobiles,
    1
)
factory(name, advanced_factory_cost, 'no', temp)

name = 'radio_factory'
temp = pm(
    name, basic_template,
    {
        glass: 0.3,
        electric_gear: 0.7
    },
    radio, 1
)
factory(name, advanced_factory_cost, 'no', temp)

name = 'telephone_factory'
temp = pm(
    name, basic_template,
    {
        glass: 0.3,
        electric_gear: 0.7
    },
    telephones, 1
)
factory(name, advanced_factory_cost, 'no', temp)

### strategic factories

name = 'electric_gear_factory'
factory(name, advanced_factory_cost, 'no', pm(
    name, basic_template,
    {
        rubber: 0.4,
        coal: 1,
        steel: 0.6
    },
    electric_gear, 1
))

name = 'machine_parts_factory'
factory(name, basic_factory_cost, 'no', pm(
    name, basic_template,
    {
        steel: 1,
        coal: 1,
    },
    machine_parts, 1
))

name = 'primitive_machine_parts_factory'
factory(name, primitive_factory_cost, 'no', pm(
    name, primitive_template,
    {
        coal: 2,
        iron: 1
    },
    machine_parts, 0.5
))

name = "synthetic_oil_factory"
factory(name, advanced_factory_cost, 'no', pm(
    name, basic_template,
    {
        coal: 4
    },
    oil, 1
))

name = "fuel_refinery"
factory(name, advanced_factory_cost, 'no', pm(
    name, basic_template,
    {
        oil: 2.5
    },
    fuel, 1
))

name = "steamer_shipyard"
coastal_factory(name, advanced_factory_cost, "no", pm(
    name, basic_template,
    {
        steel: 1,
        coal: 4,
        machine_parts: 1,
    },
    steamer_convoy, 1
))

name = "dye_factory"
factory(name, advanced_factory_cost, "no", pm(
    name, basic_template,
    {
        coal: 2
    },
    dye, 1
))

## luxury factories

name = "luxury_clothes_factory"
factory(name, basic_factory_cost, "no", pm(
    name, basic_template,
    {
        regular_clothes: 1,
        silk: 1
    },
    luxury_clothes, 1
))

name = "luxury_furniture_factory"
factory(name, basic_factory_cost, "no", pm(
    name, basic_template,
    {
        furniture: 1,
        tropical_wood: 1
    },
    luxury_furniture, 1
))

## medium factories

name = "steel_factory"
factory(name, basic_factory_cost, "no", pm(
    name, basic_template,
    {
        iron: 1,
        coal: 1
    },
    steel, 1
))

name = "artillery_factory"
factory(name, basic_factory_cost, "no", pm(
    name, basic_template,
    {
        steel: 1,
        lumber: 1
    },
    artillery, 1
))

name = "clipper_shipyard"
factory(name, basic_factory_cost, "no", pm(
    name, basic_template,
    {
        fabric: 1,
        lumber: 1,
        steel: 1
    },
    clipper_convoy, 1
))

name = "small_arms_factory"
factory(name, basic_factory_cost, "no", pm(
    name, basic_template,
    {
        steel: 1.5
    },
    small_arms, 1
))

## common factories

name = "furniture_factory"
factory(name, basic_factory_cost, "no", pm(
    name, basic_template,
    {
        lumber: 1.5
    },
    furniture, 1
))

name = "paper_mill"
factory(name, basic_factory_cost, "no", pm(
    name, basic_template,
    {
        lumber: 1.2
    },
    paper, 1
))

name = "regular_clothes_factory"
factory(name, basic_factory_cost, "no", pm(
    name, basic_template,
    {
        fabric: 2
    },
    regular_clothes, 1
))

name = "explosives_factory"
factory(name, basic_factory_cost, "no", pm(
    name, basic_template,
    {
        fertilizer: 1,
        ammunition: 0.5
    },
    explosives, 1
))

## cheap factories

name = "ammunition_factory"
factory(name, primitive_factory_cost, "no", pm(
    name, primitive_template,
    {
        sulphur: 1,
        steel: 1
    },
    ammunition, 1
))

name = "liquor_distillery"
factory(name, primitive_factory_cost, "no", pm(
    name, primitive_template,
    {
        grain: 1,
        glass: 1
    },
    liquor, 1
))

name = "liquor_distillery_fruits"
factory(name, primitive_factory_cost, "no", pm(
    name, primitive_template,
    {
        fruit: 1,
        glass: 1
    },
    liquor, 1
))

name = "winery"
factory(name, primitive_factory_cost, "no", pm(
    name, primitive_template,
    {
        fruit: 3,
        glass: 1
    },
    wine, 1
))

## basic factories

name = "cement_factory"
factory(name, primitive_factory_cost, "no", pm(
    name, primitive_template,
    {
        coal: 2,
    },
    cement, 1
))

name = "glass_factory"
factory(name, primitive_factory_cost, "no", pm(
    name, primitive_template,
    {
        coal: 1,
    },
    glass, 1
))

name = "fertilizer_factory"
factory(name, primitive_factory_cost, "no", pm(
    name, primitive_template,
    {
        sulphur: 1.2
    },
    fertilizer, 1
))



In [30]:
print_pt_templates()
print_pms()

factory_dye = {
	efficiency = 	{
		cement = 0.5
		machine_parts = 0.05
		dye = 2.0
	}
	owner = 	{
		poptype = capitalists
		effect = input
		effect_multiplier = -2.5
	}
	employees = {
	{
		poptype = craftsmen
		effect = throughput
		amount = 0.8
	}
	{
		poptype = clerks
		effect = output
		effect_multiplier = 1.25
		amount = 0.2
	}
	}
	type = factory
	workforce = 10000
}
factory_template = {
	efficiency = 	{
		cement = 0.5
		machine_parts = 0.05
	}
	owner = 	{
		poptype = capitalists
		effect = input
		effect_multiplier = -2.5
	}
	employees = {
	{
		poptype = craftsmen
		effect = throughput
		amount = 0.8
	}
	{
		poptype = clerks
		effect = output
		effect_multiplier = 1.25
		amount = 0.2
	}
	}
	type = factory
	workforce = 10000
}
cheap_factory_template = {
	efficiency = 	{
		iron = 0.5
		timber = 0.5
	}
	owner = 	{
		poptype = capitalists
		effect = input
		effect_multiplier = -2.5
	}
	employees = {
	{
		poptype = craftsmen
		effect = throughput
		amount = 0.8
	}
	{
		poptype = clerks

In [31]:
print_artisans()

artisan_fabric_factory_wool = {
	input_goods = 	{
		wool = 10
	}
	output_goods = fabric
	value = 10
	owner = 	{
		poptype = artisans
		effect = output
	}
	type = artisan
	workforce = 10000
}
artisan_fabric_factory = {
	input_goods = 	{
		cotton = 10
	}
	output_goods = fabric
	value = 10
	owner = 	{
		poptype = artisans
		effect = output
	}
	type = artisan
	workforce = 10000
}
artisan_fabric_factory_silk = {
	input_goods = 	{
		silk = 10
	}
	output_goods = fabric
	value = 10
	owner = 	{
		poptype = artisans
		effect = output
	}
	type = artisan
	workforce = 10000
}
artisan_canned_food_factory = {
	input_goods = 	{
		cattle = 10
		iron = 10
	}
	output_goods = canned_food
	value = 10
	owner = 	{
		poptype = artisans
		effect = output
	}
	type = artisan
	workforce = 10000
}
artisan_canned_food_factory_fish = {
	input_goods = 	{
		fish = 10
		iron = 10
	}
	output_goods = canned_food
	value = 10
	owner = 	{
		poptype = artisans
		effect = output
	}
	type = artisan
	workforce = 10000
}
artisan

In [32]:
print_factories()

fabric_factory_wool = {
	type = factory
	on_completion = factory
	completion_size = 0.2
	max_level = 256
	goods_cost = 	{
		cement = 10
		machine_parts = 10
	}
	time = 30
	visibility = yes
	onmap = no
	default_enabled = no
	production_type = fabric_factory_wool
	pop_build_factory = yes
}
fabric_factory = {
	type = factory
	on_completion = factory
	completion_size = 0.2
	max_level = 256
	goods_cost = 	{
		cement = 10
		machine_parts = 10
	}
	time = 30
	visibility = yes
	onmap = no
	default_enabled = no
	production_type = fabric_factory
	pop_build_factory = yes
}
fabric_factory_silk = {
	type = factory
	on_completion = factory
	completion_size = 0.2
	max_level = 256
	goods_cost = 	{
		cement = 10
		machine_parts = 10
	}
	time = 30
	visibility = yes
	onmap = no
	default_enabled = no
	production_type = fabric_factory_silk
	pop_build_factory = yes
}
canned_food_factory = {
	type = factory
	on_completion = factory
	completion_size = 0.2
	max_level = 256
	goods_cost = 	{
		cement = 10
		machi